In [1]:
# okay, first get an idea of how many tokens are there
# why it is being counted, figure out the max list you can send to API. 
# the max number of tokens you can send to the API is 8000

import pandas as pd 
import os 


folder_year = r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder"

df = pd.read_csv(os.path.join(folder_year, "dataframe.csv"))


In [2]:
df.head(3)

# abstract problem 

,publication_title,publication_num,publication_date,application_type,classifications,inventors,abstracts,descriptions,claims,publication_number,ipc_classifications,national_classifications,abstract,description
0,Anthurium andreanum plant named 'Anthbondem',NaN,20050106,utility,NaN,"[{'last_name': 'Dijk', 'first_name': 'Jan'}]",NaN,NaN,['1\n. A new and distinct\nAnthurium andreanum...,20050005336,['\n07\nG11C019/08\nA01H005/00\n'],['\nUS\nPLT365000\n'],\nA new and distinct cultivar of Anthurium and...,\n\nLATIN NAME OF THE GENUS AND SPECIES OF THE...
1,Novelty jeans,NaN,20050106,utility,NaN,"[{'last_name': 'Goldkind', 'first_name': 'Tina'}]",NaN,NaN,['1\n. A novelty jeans formed of suitable fabr...,20050000001,['\n07\nA41D001/06\n'],['\nUS\n002227000\n'],\nNovelty play jeans for children which includ...,\n\nBACKGROUND OF THE INVENTION \n The inventi...
2,Garment featuring means for temporarily attach...,NaN,20050106,utility,NaN,"[{'last_name': 'Levy', 'first_name': 'Philippe...",NaN,NaN,"[""1\n. A temporary means for attaching a cloth...",20050000002,['\n07\nA41D027/08\n'],['\nUS\n002244000\n'],"\nA garment, such as a skirt, culottes, dress,...",\n\nFIELD OF THE INVENTION \n This invention c...


In [3]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


for txt_col in ["abstract", "description", "claims"]:
    # count tokens
    df[f"{txt_col}_tokens"] = df[txt_col].fillna("").apply(lambda x: num_tokens_from_string(x, "text-embedding-3-small"))

    # count characters
    df[f"{txt_col}_characters"] = df[txt_col].fillna("").apply(lambda x: len(x))

    print(df[f"{txt_col}_tokens"].max())
    print(df[f"{txt_col}_characters"].max())



738
3034


KeyboardInterrupt: 

In [ ]:
# get a sense of the distribution of tokens. i.e. the percentiles
for txt_col in ["abstract", "description", "claims"]:
    print(f"Percentiles for {txt_col}:")
    print(df[f"{txt_col}_tokens"].quantile([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]))


total_tokens = df["abstract_tokens"].sum() + df["description_tokens"].sum() + df["claims_tokens"].sum()
print("Total number of tokens:", total_tokens)
# abstract, no problem
# description, 68877.19!! that means it is like 9 times the max number of tokens you can send to the API
# claims, on the edge... 8000 tokens.
    

# try VPN + gemini to get infinite tokens
# try CLIP finetuned to do images. Finetune CLIP in a good simple way. Just get the batche4s and ensure you are indeed getting better diagonals. Should not be too hard.


Percentiles for abstract:
0.10     57.00
0.25     89.00
0.50    127.00
0.75    167.00
0.90    205.00
0.95    236.00
0.99    318.07
Name: abstract_tokens, dtype: float64
Percentiles for description:
0.10     2389.30
0.25     3943.50
0.50     6614.00
0.75    11634.75
0.90    20822.40
0.95    30598.20
0.99    68877.19
Name: description_tokens, dtype: float64
Percentiles for claims:
0.10     371.00
0.25     691.00
0.50    1135.00
0.75    1803.50
0.90    2839.00
0.95    4003.05
0.99    8509.38
Name: claims_tokens, dtype: float64
Total number of tokens: 140167455


In [12]:
# IT SEEMS THAT THERE ARE MANY NaN VALUES.


non_string_elements = [element for element in df["description"] if not isinstance(element, str)]
print(len(non_string_elements))
print(set(non_string_elements))

df.shape

# print the titles of the patents that have NaN values in the description
df[df["description"].isna()].head(4)
# maybe is a problem with decompression of files? Focus on 20050002897
# the file is here "C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder\I20050106\UTIL0002\US20050002861A1-20050106\US20050002861A1-20050106.XML"


414
{nan}


,publication_title,publication_num,publication_date,application_type,classifications,inventors,abstracts,descriptions,claims,publication_number,ipc_classifications,national_classifications,abstract,description,abstract_tokens,abstract_characters
2860,NaN,NaN,20050106,NaN,NaN,[],NaN,NaN,[],20050002861,[],[],NaN,NaN,0,0
2865,NaN,NaN,20050106,NaN,NaN,[],NaN,NaN,[],20050002865,[],[],NaN,NaN,0,0
2876,NaN,NaN,20050106,NaN,NaN,[],NaN,NaN,[],20050002875,[],[],NaN,NaN,0,0
2899,NaN,NaN,20050106,NaN,NaN,[],NaN,NaN,[],20050002897,[],[],NaN,NaN,0,0


In [24]:
from step_1_create_dataframe import parse_xml
xml_path = r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder\I20050106\UTIL0002\US20050002861A1-20050106\US20050002861A1-20050106.XML"
parsed_xml = parse_xml(xml_path)

print(parsed_xml["publication_number"])
# df = pd.concat([df, pd.DataFrame([parsed_xml])],
                # ignore_index=True)

df[df["publication_number"] == 20050002861]

# check if all NaN values have a publication number
print(df[df["description"].isna()]["publication_number"].isna().sum())

# count the number of unique publication numbers
print(df["publication_number"].nunique())

#count the number of duplicated publication numbers
print(df["publication_number"].duplicated().sum())

# get a set of the duplicated publication numbers
duplicated_publication_numbers = set(df[df["publication_number"].duplicated()]["publication_number"].tolist())

# compare that list with the publication numbers of the NaN values. 
# if they are the same, then the NaN values are due to duplicated publication numbers
print(duplicated_publication_numbers == set(df[df["description"].isna()]["publication_number"].tolist()))

# okay, you have not figured out yet what caused it. 
# Count the number of folders.
# Run step 1 again. 

# print(df.shape)
# print(414+10980)

20050002861
0
10980
414
True


In [ ]:
# count the number of folders

In [18]:
parsed_xml

{'publication_title': 'Enhanced scintigraphic imaging agents for imaging of infection and inflammation',
 'publication_number': '20050002861',
 'publication_date': '20050106',
 'application_type': 'utility',
 'ipc_classifications': ['\n07\nA61K051/00\n'],
 'national_classifications': ['\nUS\n424001690\n'],
 'inventors': [{'last_name': 'Krause', 'first_name': 'Sabine'},
  {'last_name': 'Manchanda', 'first_name': 'Raiesh'}],
 'abstract': '\nThe invention describes enhanced scintigraphic imaging agents that can be used to localize infection and inflammation in a mammal. Specifically, the invention relates to radiolabeled, preferably technetium-99m labeled scintigraphic imaging agents that are compositions of a polysulfated glycan or mixture thereof and a compound comprising a polybasic peptide covalently linked to a radiolabel binding moiety. Methods and kits for making such compositions, and methods for using such compositions to image sites of infection and inflammation in a mammalian b

In [ ]:

"""
price gemini vs price GPT4
max tokens 


# gemini

Embeddings for Multimodal: Text 	Generate embeddings using text as an input 	Text 	Embeddings 	$0.0002 / 1k characters input
Embeddings for Multimodal: Image 	Generate embeddings using image as an input 	Image 	Embeddings 	$0.0001 / image input

gpt 4
Model	Usage
text-embedding-3-small	$0.02 / 1M tokens
text-embedding-3-large	$0.13 / 1M tokens
ada v2	$0.10 / 1M tokens

"""

# 1 token = 4 characters. quick estimation: 

# 0.0002 $ / 1k characters 
# 0.02 $ / 1M tokens


gemini_price_per_char = 0.0002 / 1000  # $0.2 per 1k characters
gpt4_price_per_1m_tokens = 0.02  # $0.02 per 1M tokens
gpt4_price_per_token = gpt4_price_per_1m_tokens / 1e6 

# 1 token = 4 characters
gemini_price_per_token = gemini_price_per_char * 4

print("Gemini price per 1M token:", 1e6 * gemini_price_per_token)
print("GPT4 price per 1M token:", 1e6 * gpt4_price_per_token)

# a bit more than 2 times more expensive... still might be doable 
# calculate average number of tokens per document

average_tokens_abstract = df["abstract_tokens"].mean()
average_tokens_description = df["description_tokens"].mean()
average_tokens_claims = df["claims_tokens"].mean()

total_average_tokens = average_tokens_abstract + average_tokens_description + average_tokens_claims
print("Average tokens per document:", total_average_tokens)

# price per document
price_per_document_gemini = total_average_tokens * gemini_price_per_token
price_per_document_gpt4 = total_average_tokens * gpt4_price_per_token

print("Price per document (Gemini):", price_per_document_gemini)
print("Price per document (GPT4):", price_per_document_gpt4)

# price per 1M documents 
price_per_1m_documents_gemini = 1e6 * price_per_document_gemini
price_per_1m_documents_gpt4 = 1e6 * price_per_document_gpt4

print("Price per 1M documents (Gemini):", price_per_1m_documents_gemini)
print("Price per 1M documents (GPT4):", price_per_1m_documents_gpt4)

# not so expensive... 

Gemini price per 1M token: 0.8
GPT4 price per 1M token: 0.02
Average tokens per document: 12301.865455502897
Price per document (Gemini): 0.009841492364402318
Price per document (GPT4): 0.0002460373091100579
Price per 1M documents (Gemini): 9841.492364402318
Price per 1M documents (GPT4): 246.03730911005792


In [ ]:

# calculate how much it would cost you again

total_number_characters = 0

# for txt_col in ["abstract", "description", "claims"]:
for txt_col in ["abstract", "description", "claims"]:
    total_number_characters += df[f"{txt_col}_characters"].sum()

print("Total number of characters:", total_number_characters)

# price for all documents 
# 0.0002 / 1k characters

price_1k_char_gemini = 0.0002
total_price_gemini =  price_1k_char_gemini * total_number_characters / 1000 
print("Total price (Gemini):", total_price_gemini)

# price per document
price_per_document_gemini = total_price_gemini / df.shape[0]
print("Price per document (Gemini):", price_per_document_gemini)

price_per_1m_documents_gemini = 1e6 * price_per_document_gemini
print("Price per 1M documents (Gemini):", price_per_1m_documents_gemini) # 10k 

# 10.600 total price for 1M documents PLUS IMAGES!!
# check if you have problems with MAX imput size with gemini!! yeah, it seems to be MAX at 2048...

# maybe have the code ready. By the time you come back, it will be cheaper with larger imput... 

# also, maybe try both? and see which one is better.
# no problem. Focus on GPT4... Start to create embeddings and do regressions

Total number of characters: 604922783
Total price (Gemini): 120.9845566
Price per document (Gemini): 0.01061826896612252
Price per 1M documents (Gemini): 10618.268966122521


In [ ]:

model="text-embedding-3-small"

In [ ]:
sentence1 = "This is a test"
sentence2 =  "second text"

# output_1_sentences = get_embeddings(sentence1, model) 
# output_2_sentences = get_embeddings([sentence1, sentence2], model) 


# output_10k_sentences = get_embeddings(" ".join(["This is a test"] * 10000), model) # every request 



In [ ]:
len(output_1_sentences)
len(output_2_sentences)
output_1_sentences[0].embedding == output_2_sentences[0].embedding


True

In [ ]:
print(output_2_sentences[0].embedding )

[0.00988506618887186, -0.005540902726352215, 0.0068014683201909065, -0.03810417652130127, -0.018254263326525688, -0.041231658309698105, -0.007651153020560741, 0.03220026567578316, 0.01892443746328354, 0.00010708322952268645, 0.05894339829683304, 0.012478001415729523, -0.023376306518912315, -0.01909995824098587, 0.02549852430820465, 0.029583394527435303, -0.08131444454193115, 0.0017113371286541224, -0.020041393116116524, 0.02832282893359661, 0.02631230652332306, -0.008736196905374527, 0.03954026475548744, 0.0029479677323251963, 0.027365438640117645, -0.042635831981897354, -0.02433369867503643, 0.002638810081407428, 0.022307218983769417, -0.055337224155664444, 0.046720702201128006, -0.03437035158276558, -0.0306524820625782, -0.012837023474276066, -0.01659478433430195, -0.00032935262424871325, -0.00602358765900135, 0.03462565690279007, -0.03357252478599548, 0.0007743900059722364, -0.036061741411685944, -0.052528876811265945, 0.005177891813218594, -0.0045715440064668655, 0.0187808275222778

In [ ]:
from call_GPT4_Chat_in_parallel import get_embedding_list
import asyncio

content_list = ["hello world!"] * 5
embeddings = asyncio.run(get_embedding_list(content_list, max_parallel_calls=500))
# check that last embedding is there
print(embeddings[-1][:10])

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
from step_2_call_GPT4_Chat_in_parallel import get_embedding_list
import asyncio
if False: 
    content_list = ["hello world!"] * 5000

    # Function to get embeddings using the existing event loop
    async def get_embeddings(content_list, batch_size):
        return await get_embedding_list(content_list, batch_size)

    # Use the existing event loop instead of asyncio.run()
    loop = asyncio.get_event_loop()
    if loop.is_running():
        embeddings = await get_embeddings(content_list, 1)
    else:
        embeddings = loop.run_until_complete(get_embeddings(content_list, 1))


In [25]:
# find why the dataframe has so many NaN values
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import html
import zipfile
import os

def parse_xml(xml_path: str) -> dict:
    """
    Parses an XML file and extracts relevant information.

    Args:
        xml_path (str): The path of the XML file.

    Returns:
        dict: A dictionary containing the extracted information.
    """

    def get_text_or_none(tag):
        return tag.get_text() 
        # return tag.get_text() if tag else None

    with open(xml_path, 'r', encoding='UTF-8') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'xml')

    # Extracting publication title, number, and date
    title = get_text_or_none(soup.find('invention-title'))
    number = get_text_or_none(
        soup.find('publication-reference').find('doc-number'))
    date = get_text_or_none(soup.find('publication-reference').find('date'))

    # Application type
    application_reference = soup.find('application-reference')
    application_type = application_reference['appl-type'] if application_reference else None

    # Classifications
    ipc_classifications = [get_text_or_none(
        ipc) for ipc in soup.find_all('classification-ipc')]
    national_classifications = [get_text_or_none(
        nc) for nc in soup.find_all('classification-national')]

    # Inventors
    inventors = [{'last_name': get_text_or_none(inventor.find('last-name')),
                  'first_name': get_text_or_none(inventor.find('first-name'))}
                 for inventor in soup.find_all('applicant')]

    # Abstract
    abstract = get_text_or_none(soup.find('abstract'))

    # Description
    description = get_text_or_none(soup.find('description'))

    # Claims
    claims = [claim.get_text(separator="\n", strip=True)
              for claim in soup.find_all('claim')]

    return {
        'publication_title': title,
        'publication_number': number,
        'publication_date': date,
        'application_type': application_type,
        'ipc_classifications': ipc_classifications,
        'national_classifications': national_classifications,
        'inventors': inventors,
        'abstract': abstract,
        'description': description,
        'claims': claims
    }




folder_year = r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder"

# create empty dataframe
df = pd.DataFrame(columns=['publication_title',
                            'publication_num',
                            'publication_date',
                            'application_type',
                            'classifications',
                            'inventors',
                            'abstract',
                            'descriptions',
                            'claims'])

# iterate over all XML files in the folder
all_xml_files = []
for root, _, files in os.walk(folder_year):
    for file in files:
        if file.lower().endswith(".xml"):
            xml_path = os.path.join(root, file)
            all_xml_files.append(xml_path)

for xml_path in tqdm(all_xml_files, desc="Parsing XML files"):
    parsed_xml = parse_xml(xml_path)
    df = pd.concat([df, pd.DataFrame([parsed_xml])],
                    ignore_index=True)




Parsing XML files:  83%|████████▎ | 861/1040 [00:34<00:07, 24.98it/s]


AttributeError: 'NoneType' object has no attribute 'get_text'

In [27]:
xml_path
okay, found the problem. there are other xml. They are SEQLST files. they should not be parsed. 

'C:\\Users\\Roberto\\Documents\\GitHub Repositories\\USPTO\\data\\fake_2005_folder\\I20050106\\UTIL0002\\US20050002861A1-20050106\\US20050002861A1-20050106-SEQLST.XML'

In [32]:
# confirm SEQLST files are all the problems 


folder_year = r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder"


# iterate over all XML files in the folder
all_xml_files = []
for root, _, files in os.walk(folder_year):
    for file in files:
        if file.lower().endswith(".xml"):
            xml_path = os.path.join(root, file)
            all_xml_files.append(xml_path)

# count the number of SEQLST files
seqlst_files = [file for file in all_xml_files if "seqlst" in file.lower()]
print(len(seqlst_files))

414


In [43]:
import pstats

# Replace 'your_module_name' with the actual name of your module
module_name = 'step_2_call_GPT4_Chat_in_parallel'

stats = pstats.Stats('profile_results')
stats.strip_dirs()  # Clean up paths for readability

# Filter stats to only include lines containing the module name
stats.fcn_list = [fn for fn in stats.stats if module_name in fn[0]]

stats.sort_stats('cumulative').print_stats()



Thu Mar 14 15:27:19 2024    profile_results

         714686 function calls (704510 primitive calls) in 2.360 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.360    1.180 runners.py:160(run)
        6    0.000    0.000    2.357    0.393 base_events.py:617(run_until_complete)
        6    0.000    0.000    2.357    0.393 windows_events.py:317(run_forever)
        6    0.000    0.000    2.357    0.393 base_events.py:593(run_forever)
      587    0.006    0.000    2.356    0.004 base_events.py:1845(_run_once)
        2    0.000    0.000    2.355    1.177 runners.py:86(run)
      587    0.001    0.000    1.745    0.003 windows_events.py:442(select)
      589    0.004    0.000    1.744    0.003 windows_events.py:804(_poll)
     1016    1.736    0.002    1.736    0.002 {built-in method _overlapped.GetQueuedCompletionStatus}
     2025    0.002    0.000    0.604    0.000 events.py:78(_run)
     20

In [44]:
import pstats
import re

stats = pstats.Stats('profile_results')

stats.strip_dirs()  # Clean up paths for readability

# Define a regular expression to match the functions or modules you're interested in
pattern = re.compile(r'\step_2_call_GPT4_Chat_in_parallel')

# Filter using the regular expression
stats.fcn_list = [fn for fn in stats.stats if pattern.search(fn[0])]

stats.sort_stats('cumulative').print_stats()


Thu Mar 14 15:27:19 2024    profile_results

         714686 function calls (704510 primitive calls) in 2.360 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.360    1.180 runners.py:160(run)
        6    0.000    0.000    2.357    0.393 base_events.py:617(run_until_complete)
        6    0.000    0.000    2.357    0.393 windows_events.py:317(run_forever)
        6    0.000    0.000    2.357    0.393 base_events.py:593(run_forever)
      587    0.006    0.000    2.356    0.004 base_events.py:1845(_run_once)
        2    0.000    0.000    2.355    1.177 runners.py:86(run)
      587    0.001    0.000    1.745    0.003 windows_events.py:442(select)
      589    0.004    0.000    1.744    0.003 windows_events.py:804(_poll)
     1016    1.736    0.002    1.736    0.002 {built-in method _overlapped.GetQueuedCompletionStatus}
     2025    0.002    0.000    0.604    0.000 events.py:78(_run)
     20

In [49]:
df = pd.read_csv(r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder\dataframe.csv")
df.head(3)

,publication_title,publication_num,publication_date,application_type,classifications,inventors,abstract,descriptions,claims,publication_number,ipc_classifications,national_classifications,description
0,Anthurium andreanum plant named 'Anthbondem',NaN,20050106,utility,NaN,"[{'last_name': 'Dijk', 'first_name': 'Jan'}]",\nA new and distinct cultivar of Anthurium and...,NaN,['1\n. A new and distinct\nAnthurium andreanum...,20050005336,['\n07\nG11C019/08\nA01H005/00\n'],['\nUS\nPLT365000\n'],\n\nLATIN NAME OF THE GENUS AND SPECIES OF THE...
1,Novelty jeans,NaN,20050106,utility,NaN,"[{'last_name': 'Goldkind', 'first_name': 'Tina'}]",\nNovelty play jeans for children which includ...,NaN,['1\n. A novelty jeans formed of suitable fabr...,20050000001,['\n07\nA41D001/06\n'],['\nUS\n002227000\n'],\n\nBACKGROUND OF THE INVENTION \n The inventi...
2,Garment featuring means for temporarily attach...,NaN,20050106,utility,NaN,"[{'last_name': 'Levy', 'first_name': 'Philippe...","\nA garment, such as a skirt, culottes, dress,...",NaN,"[""1\n. A temporary means for attaching a cloth...",20050000002,['\n07\nA41D027/08\n'],['\nUS\n002244000\n'],\n\nFIELD OF THE INVENTION \n This invention c...


In [6]:
from step_1_create_dataframe import count_tokens, MODEL, TXT_COLUMNS
import pandas as pd
import os 


# fake folder to do testing:
folder_year = r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder"

df = pd.read_csv(os.path.join(folder_year, "dataframe.csv"))

# count characters and tokens
for col in TXT_COLUMNS:
    df[f"{col}_characters"] = df[col].apply(lambda x: len(x))
    df[f"{col}_tokens"] = df[col].apply(lambda x: count_tokens(x, MODEL))

KeyError: 'title'

In [8]:
# save the dataframe
df.to_csv(os.path.join(folder_year, "dataframe.csv"), index=False)

In [11]:
# # open dataframe
# df.to_csv("data/df_with_embeddings.csv", index=False)
# print("Dataframe saved")



In [12]:
df_embeddings

,publication_title,publication_num,publication_date,application_type,classifications,inventors,abstract,descriptions,claims,publication_number,ipc_classifications,national_classifications,description,abstract_characters,abstract_tokens,claims_characters,claims_tokens,description_characters,description_tokens,abstract_embeddings
0,Anthurium andreanum plant named 'Anthbondem',NaN,20050106,utility,NaN,"[{'last_name': 'Dijk', 'first_name': 'Jan'}]",\nA new and distinct cultivar of Anthurium and...,NaN,['1\n. A new and distinct\nAnthurium andreanum...,20050005336,['\n07\nG11C019/08\nA01H005/00\n'],['\nUS\nPLT365000\n'],\n\nLATIN NAME OF THE GENUS AND SPECIES OF THE...,643,129,112,29,9258,2306,"[[-0.024430248886346817, 0.0164019837975502, 0..."
1,Novelty jeans,NaN,20050106,utility,NaN,"[{'last_name': 'Goldkind', 'first_name': 'Tina'}]",\nNovelty play jeans for children which includ...,NaN,['1\n. A novelty jeans formed of suitable fabr...,20050000001,['\n07\nA41D001/06\n'],['\nUS\n002227000\n'],\n\nBACKGROUND OF THE INVENTION \n The inventi...,507,104,6585,1441,23051,5122,"[[-0.019678520038723946, -0.000915646378416568..."
2,Garment featuring means for temporarily attach...,NaN,20050106,utility,NaN,"[{'last_name': 'Levy', 'first_name': 'Philippe...","\nA garment, such as a skirt, culottes, dress,...",NaN,"[""1\n. A temporary means for attaching a cloth...",20050000002,['\n07\nA41D027/08\n'],['\nUS\n002244000\n'],\n\nFIELD OF THE INVENTION \n This invention c...,744,149,2582,541,11689,2466,"[[-0.03638230636715889, 0.02144351229071617, 0..."
3,Under pant garment comfort strip,NaN,20050106,utility,NaN,"[{'last_name': 'Bushelman', 'first_name': 'Sus...",\nThe invention is a comfort strip for control...,NaN,['1\n. A comfort strip for controlling under p...,20050000003,['\n07\nA41B009/00\n'],['\nUS\n002400000\n'],\n\nTECHNICAL FIELD \n The invention relates t...,139,28,3294,757,18160,3887,"[[-0.019858578220009804, 0.009553547948598862,..."
4,Integrated toilet and air deodorizer,NaN,20050106,utility,NaN,"[{'last_name': 'Yun', 'first_name': 'Danny'}]","\nIn one embodiment, an integrated toilet and ...",NaN,['1\n. An integrated toilet and air deodorizer...,20050000004,['\n07\nE03D009/04\n'],['\nUS\n004216000\n'],\n\nBACKGROUND \n 1. Field \n The present inve...,710,149,3650,814,27471,5826,"[[-0.017842788249254227, 0.03828040510416031, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Backpack with incorporated gun scabbard,NaN,20050106,utility,NaN,"[{'last_name': 'Eberle', 'first_name': 'Glen'}]",\nA method of forming a backpack with an integ...,NaN,"['1\n. A backpack comprising, in combination:\...",20050000995,['\n07\nA45F003/04\n'],['\nUS\n224637000\n224638000\n224643000\n22464...,\n\nCROSS-REFERENCE TO RELATED APPLICATIONS \n...,675,135,6390,1427,30953,6411,"[[-0.005150278564542532, 0.024220596998929977,..."
996,Carrying case for portable electronic device,NaN,20050106,utility,NaN,"[{'last_name': 'Petravic', 'first_name': 'Robi...",\nA carrying case for a portable electronic de...,NaN,['1\n. A carrying case for a portable electron...,20050000996,['\n07\nA45F003/04\n'],['\nUS\n224642000\n'],\n\nSUMMARY OF THE INVENTION \n The present in...,415,74,6457,1407,13164,2901,"[[-0.0006410701316781342, 0.03963496536016464,..."
997,Plant for chemical treatment of a surface of a...,NaN,20050106,utility,NaN,"[{'last_name': 'Koza', 'first_name': 'Walter'}...",\nA plant is provided for chemical treatment o...,NaN,['1\n. An assembly for chemical treatment of t...,20050000997,['\n07\nB65H023/02\n'],['\nUS\n226018000\n'],\n\nCROSS-REFERENCE TO RELATED APPLICATION \n ...,830,155,4992,1028,10069,2059,"[[-0.016492094844579697, 0.007753642275929451,..."
998,Hand-held working tool,NaN,20050106,utility,NaN,"[{'last_name': 'Grazioli', 'first_name': 'Mari...",\nThe present invention relates to a hand-held...,NaN,"['1\n. Hand-held working tool, such as one of ...",20050000998,['\n07\nB25C001/14\n'],['\nUS\n227002000\n22701

In [9]:
# find bug in code 
from step_1_create_dataframe import parse_xml
from bs4 import BeautifulSoup
xml_path = r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder\I20050106\UTIL0000\US20050000001A1-20050106\US20050000001A1-20050106.XML"
# parsed_xml = parse_xml(xml_path)
# print(parsed_xml)
with open(xml_path, 'r', encoding='UTF-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'xml')
application_id = soup.find('us-related-documents').find('doc-number').get_text()
print(application_id)
# 10098856

10098856
